In [1]:
import pandas as pd
import numpy as np
from scipy.special import expit #Vectorized sigmoid function
from scipy import optimize

In [2]:
data = pd.read_csv("Original dataset.csv")

# Data Wrangling

In [3]:
data.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST_CITY_NAME,DEST_STATE_NM,ARR_DEL15,DIVERTED,Unnamed: 11
0,2017,1,1,7,6,"Boston, MA",Massachusetts,"Los Angeles, CA",California,1.0,0.0,NaN
1,2017,1,1,8,7,"Boston, MA",Massachusetts,"Los Angeles, CA",California,0.0,0.0,NaN
2,2017,1,1,9,1,"Boston, MA",Massachusetts,"Los Angeles, CA",California,1.0,0.0,NaN
3,2017,1,1,10,2,"Boston, MA",Massachusetts,"Los Angeles, CA",California,0.0,0.0,NaN
4,2017,1,1,11,3,"Boston, MA",Massachusetts,"Los Angeles, CA",California,1.0,0.0,NaN


In [4]:
data.shape

(450017, 12)

In [5]:
data.dtypes

YEAR                  int64
QUARTER               int64
MONTH                 int64
DAY_OF_MONTH          int64
DAY_OF_WEEK           int64
ORIGIN_CITY_NAME     object
ORIGIN_STATE_NM      object
DEST_CITY_NAME       object
DEST_STATE_NM        object
ARR_DEL15           float64
DIVERTED            float64
Unnamed: 11         float64
dtype: object

In [6]:
data.isnull().sum()

YEAR                     0
QUARTER                  0
MONTH                    0
DAY_OF_MONTH             0
DAY_OF_WEEK              0
ORIGIN_CITY_NAME         0
ORIGIN_STATE_NM          0
DEST_CITY_NAME           0
DEST_STATE_NM            0
ARR_DEL15            10372
DIVERTED                 0
Unnamed: 11         450017
dtype: int64

In [7]:
# I delete the "Unnamed: 11" column as I will not use it
keep_cols = ["YEAR", "QUARTER", "MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN_CITY_NAME", "ORIGIN_STATE_NM",
             "DEST_CITY_NAME", "DEST_STATE_NM", "DIVERTED", "ARR_DEL15"]
data = data[keep_cols]

In [8]:
# Find non-numerical columns
obj_columns = data.select_dtypes(['object']).columns
# Transform all their rows in categorical variables
for i in obj_columns:
    data[i] = data[i].astype('category')
# Transform categorical variables into numerical ones   
data[obj_columns] = data[obj_columns].apply(lambda x: x.cat.codes)

In [9]:
data.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST_CITY_NAME,DEST_STATE_NM,DIVERTED,ARR_DEL15
0,2017,1,1,7,6,36,19,167,4,0.0,1.0
1,2017,1,1,8,7,36,19,167,4,0.0,0.0
2,2017,1,1,9,1,36,19,167,4,0.0,1.0
3,2017,1,1,10,2,36,19,167,4,0.0,0.0
4,2017,1,1,11,3,36,19,167,4,0.0,1.0


Separating the training and test set

In [10]:
# The training set is the one that doesn't contain missing values for the "ARR_DEL15" column
train = data[(data['ARR_DEL15'] == 1) | (data['ARR_DEL15'] == 0)]

# The remaining rows are the test set
test = data[pd.isnull(data['ARR_DEL15'])]
# I also delete the "ARR_DEL15" columns
keep_cols = ["YEAR", "QUARTER", "MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN_CITY_NAME", "ORIGIN_STATE_NM",
             "DEST_CITY_NAME", "DEST_STATE_NM", "DIVERTED"]
test = test[keep_cols]

In [11]:
train.to_csv("new_train.csv")
test.to_csv("new_test.csv")

# Building the model

In [12]:
datafile = 'new_train.csv'
# Load file using numpy
cols = np.loadtxt(datafile, delimiter=',', unpack=True, usecols=range(1,12), skiprows=1)
# Form the "X" matrix (with the features) and "y" vector (with the labels)
X = np.transpose(np.array(cols[:-1]))
y = np.transpose(np.array(cols[-1:]))
m = y.size # number of training examples

# Insert a column of 1's into the "X" matrix,
# so that when we multiply the features by the coefficients of each one,
# there is no feature that multiplies the "intercept" coefficient
X = np.insert(X,0,1,axis=1)

In [13]:
#Divide the sample into two: ones with positive classification (delay), one with null classification (no delay)
pos = np.array([X[i] for i in xrange(X.shape[0]) if y[i] == 1])
neg = np.array([X[i] for i in xrange(X.shape[0]) if y[i] == 0])

In [14]:
#Hypothesis function and cost function for logistic regression
def h(mytheta,myX): #Logistic hypothesis function
    return expit(np.dot(myX,mytheta))

#Cost function, default lambda (regularization) 0
def computeCost(mytheta,myX,myy): 
    term1 = np.dot(-np.array(myy).T,np.log(h(mytheta,myX)))
    term2 = np.dot((1-np.array(myy)).T,np.log(1-h(mytheta,myX)))
    regterm = np.sum(np.dot(mytheta[1:].T,mytheta[1:])) #Skip theta0
    return float( (1./m) * ( np.sum(term1 - term2) + regterm ) )

In [15]:
# I use some scipy.optimize function called "fmin".
# "fmin" does not need to be told explicitly the derivative terms.
# It only needs the cost function, and it minimizes it with the "downhill simplex algorithm."

def optimizeTheta(mytheta,myX,myy):
    result = optimize.fmin(computeCost, x0=mytheta, args=(myX, myy), maxiter=1000, full_output=True)
    return result[0], result[1]

In [16]:
# Initialize the values of theta
initial_theta = np.zeros((X.shape[1],1))
computeCost(initial_theta,X,y)

0.6931471805600717

In [17]:
# theta = coefficients
# mincost = value of the minimized cost function
theta, mincost = optimizeTheta(initial_theta,X,y)

Optimization terminated successfully.
         Current function value: 0.515211
         Iterations: 771
         Function evaluations: 1093


In [18]:
def makePrediction(mytheta, myx):
    return h(mytheta,myx) >= 0.5

#Compute the percentage of the correct estimations on training samples:
pos_correct = float(np.sum(makePrediction(theta,pos)))
neg_correct = float(np.sum(np.invert(makePrediction(theta,neg))))
tot = len(pos)+len(neg)
prcnt_correct = float(pos_correct+neg_correct)/tot
print "Fraction of training samples correctly predicted: %f." % prcnt_correct

Fraction of training samples correctly predicted: 0.777778.


# Making predictions

In [19]:
datafile = 'new_test.csv'
# Load file using numpy
cols = np.loadtxt(datafile,delimiter=',',unpack=True, usecols=range(1,11), skiprows=1)

# Form the same "X" matrix and "y" vector as before (but for the test set this time)
X_test = np.transpose(np.array(cols[:]))
X_test = np.insert(X_test,0,1,axis=1)

In [20]:
predictions = []
for i in X_test:
    predictions.append(h(theta,i)*100)

In [21]:
delay_prob = pd.DataFrame(np.array(predictions).reshape(10372,1), columns = ["ARR_DEL15"])

In [22]:
delay_prob

,ARR_DEL15
0,27.930235
1,25.983674
2,27.109845
3,32.294696
4,35.361830
5,25.983674
6,14.764565
7,25.758170
8,30.282395
9,17.020461
